In [ ]:


# OSCT Deep Learning Workshop_CNN: 2023
     

# Case study: medical image segmentation
     

# Mounting google drive

from google.colab import drive
drive.mount('/content/gdrive')
     
import os

directory = '/content/gdrive/My Drive/KneeJointSpaceSegmentationDataset/results/pred_train'
os.makedirs(directory, exist_ok=True)

directory = '/content/gdrive/My Drive/KneeJointSpaceSegmentationDataset/results/X_train'
os.makedirs(directory, exist_ok=True)

directory = '/content/gdrive/My Drive/KneeJointSpaceSegmentationDataset/results/Y_train'
os.makedirs(directory, exist_ok=True)

directory = '/content/gdrive/My Drive/KneeJointSpaceSegmentationDataset/results/Y_val'
os.makedirs(directory, exist_ok=True)

directory = '/content/gdrive/My Drive/KneeJointSpaceSegmentationDataset/results/X_val'
os.makedirs(directory, exist_ok=True)

# Pre-processing steps 

import numpy as np
import os
from itertools import chain
from skimage.io import imread, imshow, concatenate_images, imread_collection


import os
os.chdir('/content/gdrive/My Drive/KneeJointSpaceSegmentationDataset')
!pwd
img_dir = './Train_X/*.png'
ground_dir=  './Train_Y/*.png'

# Training images
ground_image_set = imread_collection(ground_dir)
ground_image_list = list(ground_image_set)

train_image_set = imread_collection(img_dir)
train_image_list = list(train_image_set)


# Saving the images in list format for further processesing in training phase

np.savez('data_train_list_apt.npz',train_image_list=train_image_list,ground_image_list=ground_image_list)


     

# Training

import numpy as np
import matplotlib
import os
import matplotlib.pyplot as plt
import  cv2
from tqdm import tqdm_notebook, tnrange
import skimage
#import scikit-image as skimage
from itertools import chain
from skimage.io import imread, imshow, concatenate_images, imread_collection
from skimage.transform import resize
from skimage.morphology import label
from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_score #, jaccard_similarity_score
import random
import tensorflow as tf

from keras import backend as K
from keras.backend import *
from keras.models import *
from numpy import load
from keras.layers import *
from keras.optimizers import *
from keras.callbacks import ModelCheckpoint, LearningRateScheduler
from keras import backend as keras
from keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from tensorflow.keras.optimizers import Adam


from tensorflow.python.client import device_lib

# GPU or CPU

# with tf.device('/cpu:0'):
with tf.device('/gpu:0'):


    im_width=224
    im_height=224

## Loading train images list

    data1= load('data_train_list_apt.npz')
    train_image_list= data1['train_image_list']


    X_list = []
    for img in train_image_list:
        img = img / 255
        X_list.append(img)

    X_array = np.asarray(X_list)
    X_array = np.reshape(X_array, (X_array.shape[0], X_array.shape[1], X_array.shape[2], 1))

    print(X_array.shape)


## Loading ground/annotated images list

   # data = load('data_list.npz', allow_pickle=True)
    data = load ('data_train_list_apt.npz', allow_pickle=True)
    ground_image_list = data['ground_image_list']

    temp_ground = ground_image_list[0:40]


    Y_list = []

    for image in ground_image_list:
        image = image[0:224, 0:224]                             ## Found some shape mismatches, thus resizing the images
        if image.shape < (im_width, im_height):
            image = np.resize(image, (im_width, im_height))
        #  image= image.resize((224,224))

        print(image.shape)
        image = image / 255

        Y_list.append(image)

    Y_array = np.asarray(Y_list)

    print("Shape fo y", Y_array.shape)

    Y_array = np.reshape(Y_array, (Y_array.shape[0], Y_array.shape[1], Y_array.shape[2], 1))

    print(Y_array.shape)


    ## U-net Architecture:


    def conv2d_block(input_tensor,n_filters,kernel_size,batchnorm=True):
        # first layer
        x = Conv2D(filters=n_filters, kernel_size=(kernel_size, kernel_size), kernel_initializer="he_normal",
                   padding="same")(input_tensor)
        if batchnorm:
            x = BatchNormalization()(x)
        x = Activation("relu")(x)
        # second layer
        x = Conv2D(filters=n_filters, kernel_size=(kernel_size, kernel_size), kernel_initializer="he_normal",
                   padding="same")(x)
        if batchnorm:
            x = BatchNormalization()(x)
        x = Activation("relu")(x)
        return x



    def get_unet(input_img, n_filters=16, dropout=0.5, batchnorm=True):
        # contracting path

        c1 = conv2d_block(input_img, n_filters=n_filters*1, kernel_size=3, batchnorm=batchnorm)
        p1 = MaxPooling2D((2,2)) (c1)
        p1 = Dropout(dropout*0.5)(p1)

        c2 = conv2d_block(p1, n_filters=n_filters*2, kernel_size=3, batchnorm=batchnorm)
        p2 = MaxPooling2D((2,2)) (c2)
        p2 = Dropout(dropout)(p2)

        
        c5 = conv2d_block(p2, n_filters=n_filters*4, kernel_size=3, batchnorm=batchnorm)

        # expansive path
        u6 = Conv2DTranspose(n_filters*8, (3, 3), strides=(2, 2), padding='same') (c5)
        u6 = concatenate([u6, c2])
        u6 = Dropout(dropout)(u6)
        c6 = conv2d_block(u6, n_filters=n_filters*8, kernel_size=3, batchnorm=batchnorm)

        u7 = Conv2DTranspose(n_filters*4, (3, 3), strides=(2, 2), padding='same') (c6)
        u7 = concatenate([u7, c1])
        u7 = Dropout(dropout)(u7)
        c7 = conv2d_block(u7, n_filters=n_filters*4, kernel_size=3, batchnorm=batchnorm)

        outputs = Conv2D(1, (1, 1), activation='sigmoid') (c7)
        model = Model(inputs=[input_img], outputs=[outputs])
        return model


    def data_split(matrix, target, test_proportion):
        ratio = int(matrix.shape[0]/test_proportion)
        X_train = matrix[ratio:,:,:]
        X_test =  matrix[:ratio,:,:]
        Y_train = target[ratio:,:,:]
        Y_test =  target[:ratio,:,:]
        return X_train, X_test, Y_train, Y_test


    # Focal loss 


    def focal_loss(gamma=2., alpha=.25):
        def focal_loss_fixed(y_true, y_pred):
            pt_1 = tf.where(tf.equal(y_true, 1), y_pred, tf.ones_like(y_pred))
            pt_0 = tf.where(tf.equal(y_true, 0), y_pred, tf.zeros_like(y_pred))
            return -K.sum(alpha * K.pow(1. - pt_1, gamma) * K.log(pt_1))-K.sum((1-alpha) * K.pow( pt_0, gamma) * K.log(1. - pt_0))
        return focal_loss_fixed


    X_train,X_val,Y_train,Y_val=data_split(X_array,Y_array,3)
    print (X_val.shape)
    print (X_train.shape)

    ## Model Executation

    input_img = Input((im_width, im_height,1), name='img')
    model = get_unet(input_img, n_filters=32, dropout=0.05, batchnorm=True)
    model.compile(optimizer=Adam(),loss=[focal_loss(alpha=.25, gamma=2)], metrics=["accuracy"])
    model.summary()


    ## Saving the best prediction moel(s)

    callbacks = [
        EarlyStopping(patience=10, verbose=1),
        ReduceLROnPlateau(factor=0.1, patience=3, min_lr=0.00001, verbose=1),
                 ModelCheckpoint('model-epoch100.h5', verbose=1, save_best_only=True, save_weights_only=True)      ## Best model gets saved as 'model-epoch100.h5'
    ]


    ## Testing the train images:

    results = model.fit(X_train, Y_train, batch_size=8, epochs=10,callbacks=callbacks,
                        validation_data=(X_val, Y_val))


     

# Test

import numpy as np
import matplotlib
import os
import matplotlib.pyplot as plt
import  cv2
from tqdm import tqdm_notebook, tnrange
import skimage
#import scikit-image as skimage
from itertools import chain
from skimage.io import imread, imshow, concatenate_images, imread_collection
from skimage. transform import resize
from skimage.morphology import label
from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_score#, jaccard_similarity_score
import random
import tensorflow as tf

from keras import backend as K
from keras.backend import *
from keras.models import *
from numpy import load
from keras.layers import *
from keras.optimizers import *
from keras.callbacks import ModelCheckpoint, LearningRateScheduler
from keras import backend as keras
from keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau


from tensorflow.python.client import device_lib

im_width = 224
im_height = 224


#data = load('data_list.npz')
data = load('data_train_list_apt.npz')
ground_image_list = data['ground_image_list']


data2= load('data_train_list_apt.npz')


train_image_list = data2['train_image_list']

print (train_image_list[0].shape)
print (ground_image_list[0].shape)

X_list = []
for img in train_image_list:
    img = img / 255
    X_list.append(img)

X_array = np.asarray(X_list)
X_array = np.reshape(X_array, (X_array.shape[0], X_array.shape[1], X_array.shape[2], 1))

print(X_array.shape)


data = load('data_train_list_apt.npz')
ground_image_list = data['ground_image_list']


Y_list = []

for image in ground_image_list:
    image = image[0:224, 0:224]
    if image.shape < (im_width, im_height):
        image = np.resize(image, (im_width, im_height))

    #print(image.shape)
    image = image / 255

    Y_list.append(image)

Y_array = np.asarray(Y_list)
Y_array = np.reshape(Y_array, (Y_array.shape[0], Y_array.shape[1], Y_array.shape[2], 1))
print(Y_array.shape)


## Train Val Split

def data_split(matrix, target, test_proportion):
    ratio = int(matrix.shape[0] / test_proportion)
    X_train = matrix[ratio:, :, :]
    X_test = matrix[:ratio, :, :]
    Y_train = target[ratio:, :, :]
    Y_test = target[:ratio, :, :]
    return X_train, X_test, Y_train, Y_test


X_train,X_val,Y_train,Y_val=data_split(X_array,Y_array,3)

print ("=====================")
print (X_val.shape)
print (X_train.shape)


## U-net Architecture:


def conv2d_block(input_tensor, n_filters, kernel_size, batchnorm=True):
    # first layer
    x = Conv2D(filters=n_filters, kernel_size=(kernel_size, kernel_size), kernel_initializer="he_normal",
               padding="same")(input_tensor)
    if batchnorm:
        x = BatchNormalization()(x)
    x = Activation("relu")(x)
    # second layer
    x = Conv2D(filters=n_filters, kernel_size=(kernel_size, kernel_size), kernel_initializer="he_normal",
               padding="same")(x)
    if batchnorm:
        x = BatchNormalization()(x)
    x = Activation("relu")(x)
    return x


def get_unet(input_img, n_filters=16, dropout=0.5, batchnorm=True):
    # contracting path

    c1 = conv2d_block(input_img, n_filters=n_filters * 1, kernel_size=3, batchnorm=batchnorm)
    p1 = MaxPooling2D((2, 2))(c1)
    p1 = Dropout(dropout * 0.5)(p1)

    c2 = conv2d_block(p1, n_filters=n_filters * 2, kernel_size=3, batchnorm=batchnorm)
    p2 = MaxPooling2D((2, 2))(c2)
    p2 = Dropout(dropout)(p2)


    c5 = conv2d_block(p2, n_filters=n_filters * 4, kernel_size=3, batchnorm=batchnorm)

    # expansive path
    u6 = Conv2DTranspose(n_filters * 8, (3, 3), strides=(2, 2), padding='same')(c5)
    u6 = concatenate([u6, c2])
    u6 = Dropout(dropout)(u6)
    c6 = conv2d_block(u6, n_filters=n_filters * 8, kernel_size=3, batchnorm=batchnorm)

    u7 = Conv2DTranspose(n_filters * 4, (3, 3), strides=(2, 2), padding='same')(c6)
    u7 = concatenate([u7, c1])
    u7 = Dropout(dropout)(u7)
    c7 = conv2d_block(u7, n_filters=n_filters * 4, kernel_size=3, batchnorm=batchnorm)

   

    outputs = Conv2D(1, (1, 1), activation='sigmoid')(c7)
    model = Model(inputs=[input_img], outputs=[outputs])
    return model


def data_split(matrix, target, test_proportion):
    ratio = int(matrix.shape[0] / test_proportion)
    X_train = matrix[ratio:, :, :]
    X_test = matrix[:ratio, :, :]
    Y_train = target[ratio:, :, :]
    Y_test = target[:ratio, :, :]
    return X_train, X_test, Y_train, Y_test


## Code for focal loss ##


def focal_loss(gamma=2., alpha=.25):
    def focal_loss_fixed(y_true, y_pred):
        pt_1 = tf.where(tf.equal(y_true, 1), y_pred, tf.ones_like(y_pred))
        pt_0 = tf.where(tf.equal(y_true, 0), y_pred, tf.zeros_like(y_pred))
        return -K.sum(alpha * K.pow(1. - pt_1, gamma) * K.log(pt_1)) - K.sum(
            (1 - alpha) * K.pow(pt_0, gamma) * K.log(1. - pt_0))

    return focal_loss_fixed


X_train, X_val, Y_train, Y_val = data_split(X_array, Y_array, 3)
print(X_val.shape)
print(X_train[0].shape)



input_img = Input((im_width, im_height,1), name='img')
model = get_unet(input_img, n_filters=32, dropout=0.05, batchnorm=True)
model.compile(optimizer=Adam(), loss=[focal_loss(alpha=.25, gamma=2)], metrics=["accuracy"])



model.load_weights('model-epoch100.h5')

pred_val = model.predict(X_val,verbose=1)
print (type(pred_val))
#np.savez('model_results.npz',preds_val= pred_val, preds_train=preds_train)
pred_train= model.predict(X_train,verbose=1)

print ("predicted!")

np.savez('results.npz',preds_val= pred_val, preds_train=pred_train, X_train= X_train, Y_train=Y_train, X_val= X_val, Y_val= Y_val)


     

# Saving files and image visualization

import numpy as np
from numpy import load
import matplotlib
import scipy
import imageio


data= load('results.npz')
#preds_val=data['preds_val']
preds_train= data['preds_train']


print ('loaded')

os.chdir('/content/gdrive/My Drive/KneeJointSpaceSegmentationDataset')

num= 0

for img in preds_train:
    img[img > 0.3] = 1
    img= np.squeeze(img)
    imageio.imwrite('./results/pred_train/ '+str(num) +'.png', img)
    num +=1


print ('done!')

flag=0

for img in X_train:
    img = np.squeeze(img)
    imageio.imwrite('./results/X_train/ ' + str(flag) + '.png', img)
    flag += 1

print ('done!')
temp= 0

for img in Y_train:
    img = np.squeeze(img)
    imageio.imwrite('./results/Y_train/ ' + str(temp) + '.png', img)
    temp += 1


print ('done!')
cal= 0

for img in Y_val:
    img = np.squeeze(img)
    imageio.imwrite('./results/Y_val/ ' + str(cal) + '.png', img)
    cal += 1

print ('done!')

j= 0

for img in X_val:
    imageio.imwrite('./results/X_val/ ' + str(j) + '.png', img)
    j += 1

print ('done!')
     

# Calculating Intersection over Union (IoU)

#import scikit-learn as sklearn
import cv2
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
from sklearn.metrics import jaccard_score

os.chdir('/content/gdrive/My Drive/KneeJointSpaceSegmentationDataset')

ground_dir= './results/Y_val/ 27.png'

ground_img= im = plt.imread(ground_dir)
print (type(ground_img))
print (ground_img.shape)

pred_dir  ='./results/pred_train/ 27.png'

pred_img=  plt.imread(pred_dir)

IOU_score = jaccard_score(np.round(ground_img), np.round(pred_img), average='micro')

print (IOU_score)
     

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).
/content/gdrive/My Drive/KneeJointSpaceSegmentationDataset
(84, 224, 224, 1)
(224, 224)
(224, 224)
(224, 224)
(224, 224)
(224, 224)
(224, 224)
(224, 224)
(224, 224)
(224, 224)
(224, 224)
(224, 224)
(224, 224)
(224, 224)
(224, 224)
(224, 224)
(224, 224)
(224, 224)
(224, 224)
(224, 224)
(224, 224)
(224, 224)
(224, 224)
(224, 224)
(224, 224)
(224, 224)
(224, 224)
(224, 224)
(224, 224)
(224, 224)
(224, 224)
(224, 224)
(224, 224)
(224, 224)
(224, 224)
(224, 224)
(224, 224)
(224, 224)
(224, 224)
(224, 224)
(224, 224)
(224, 224)
(224, 224)
(224, 224)
(224, 224)
(224, 224)
(224, 224)
(224, 224)
(224, 224)
(224, 224)
(224, 224)
(224, 224)
(224, 224)
(224, 224)
(224, 224)
(224, 224)
(224, 224)
(224, 224)
(224, 224)
(224, 224)
(224, 224)
(224, 224)
(224, 224)
(224, 224)
(224, 224)
(224, 224)
(224, 224)
(224, 224)
(224, 224)
(224, 224)
(224, 224)
(224, 224)
(224, 224)


1/1 [==============================] - 0s 230ms/step
<class 'numpy.ndarray'>
2/2 [==============================] - 0s 454ms/step
predicted!


loaded


done!


done!


done!


done!


done!
<class 'numpy.ndarray'>
(224, 224)
0.04442662389735365
